# Assignment 4: Quantum Channel Classification
We reuse the Week 3 classifier to tag noisy channels quickly during calibration checks.

This week we focus on applying the saved classifier and showing that the workflow runs end to end.

**Task plan**
1. Explain why quick channel labels matter and list the workflow you will follow.
2. Load the helper code, pull in the trained estimator, and note any backup plan.
3. Rebuild the feature mapper from Kraus operators to Choi vectors.
4. Test the model on a small batch of synthetic channels and record observations.

## Background notes
- A quantum channel $\mathcal{E}$ is a completely positive, trace-preserving map. We write $\mathcal{E}(\rho) = \sum_k K_k \rho K_k^\dagger$ with $\sum_k K_k^\dagger K_k = I$ so that each Kraus operator $K_k$ captures one noise branch.
- The Choi matrix $C_{\mathcal{E}} = (\mathcal{E} \otimes I)(|\Phi^+\rangle\langle\Phi^+|)$ models how the channel acts on half of an entangled pair. Flattening its real and imaginary parts gives a steady feature vector.
- Classification is lighter than full tomography. We only emit labels like depolarising or amplitude damping, which keeps the calibration loop fast.

## Task 1 · Environment check
- Confirm qiskit, numpy, pandas, joblib, and scikit-learn import without errors.
- If anything is missing, run the pip cell below and log the command in your notes.
- Once the imports work, move to Task 2.

In [ ]:
# Log your environment status here once Task 1 is complete.


In [3]:
# Install prerequisites if the kernel is missing a package.
%pip install qiskit scikit-learn joblib pandas


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


## Task 2 · Import helper modules
- Run the cell below to pull in numpy, joblib, qiskit, pandas, and os.
- Keep the imports in one place so later tasks stay consistent.

In [1]:
import numpy as np
import joblib
from sklearn.dummy import DummyClassifier
from qiskit.quantum_info import Kraus, Choi
import pandas as pd
import os


/var/folders/39/0cxygs9543l3tmkygnchf2gc0000gn/T/ipykernel_6265/804130532.py:4: DeprecationWarning: Using Qiskit with Python 3.9 is deprecated as of the 2.1.0 release. Support for running Qiskit with Python 3.9 will be removed in the 2.3.0 release, which coincides with when Python 3.9 goes end of life.
  from qiskit.quantum_info import Kraus, Choi


In [15]:
def kraus_to_choi(kraus_ops):
    """
    Convert list of Kraus operators into Choi matrix.
    """
    import numpy as np
    
    d = kraus_ops[0].shape[0]
    
    # Create maximally entangled state |Phi+>
    phi = np.zeros((d*d, 1), dtype=complex)
    for i in range(d):
        phi[i*d + i, 0] = 1
    phi /= np.sqrt(d)
    
    choi = np.zeros((d*d, d*d), dtype=complex)

    for K in kraus_ops:
        K_ext = np.kron(K, np.eye(d))
        choi += K_ext @ (phi @ phi.conj().T) @ K_ext.conj().T

    return choi

In [16]:
def channel_to_feature(kraus_ops):
    choi = kraus_to_choi(kraus_ops)
    
    return np.concatenate([
        choi.real.flatten(),
        choi.imag.flatten()
    ])

## Task 3 · Build a calibration-time classifier
- Implement a lightweight classifier that maps Choi features to channel labels without relying on saved artefacts.
- Keep the training code inside the provided function so reviewers can see your modelling choices.
- You may reuse utilities from earlier assignments (data loaders, feature encoders) as long as they are imported inside the function.

In [3]:
def build_channel_classifier():
    """
    Train and return a classifier that distinguishes
    depolarizing and amplitude damping channels.
    """
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import classification_report
    
    np.random.seed(42)

    features = []
    labels = []

    # Generate depolarizing channels
    for p in np.linspace(0.01, 0.9, 50):
        ch = depolarizing_kraus(p)
        features.append(channel_to_feature(ch))
        labels.append("depolarizing")

    # Generate amplitude damping channels
    for gamma in np.linspace(0.01, 0.9, 50):
        ch = amplitude_damping_kraus(gamma)
        features.append(channel_to_feature(ch))
        labels.append("amplitude_damping")

    X = np.array(features)
    y = np.array(labels)

    # Split data
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # Train classifier
    model = LogisticRegression(max_iter=500)
    model.fit(X_train, y_train)

    # Evaluate
    y_pred = model.predict(X_val)
    print("Validation Performance:")
    print(classification_report(y_val, y_pred))

    return model

## Task 4 · Build channel features
- Regenerate the Kraus operators you used during training or adapt them for this demo.
- Ensure `channel_to_feature` outputs the same ordering the model expects (real part first, imaginary part second).

In [2]:
I2 = np.eye(2, dtype=complex)
X2 = np.array([[0, 1], [1, 0]], dtype=complex)
Y2 = np.array([[0, -1j], [1j, 0]], dtype=complex)
Z2 = np.array([[1, 0], [0, -1]], dtype=complex)

def depolarizing_kraus(p):
    k0 = np.sqrt(1 - p) * I2
    k1 = np.sqrt(p/3) * X2
    k2 = np.sqrt(p/3) * Y2
    k3 = np.sqrt(p/3) * Z2
    return Kraus([k0, k1, k2, k3])

def amplitude_damping_kraus(gamma):
    k0 = np.array([[1, 0], [0, np.sqrt(1-gamma)]], dtype=complex)
    k1 = np.array([[0, np.sqrt(gamma)], [0, 0]], dtype=complex)
    return Kraus([k0, k1])

def channel_to_feature(channel):
    choi = Choi(channel).data
    feat = np.concatenate([choi.real.flatten(), choi.imag.flatten()])
    return feat


In [4]:
model = build_channel_classifier()

Validation Performance:
                   precision    recall  f1-score   support

amplitude_damping       0.80      1.00      0.89         8
     depolarizing       1.00      0.83      0.91        12

         accuracy                           0.90        20
        macro avg       0.90      0.92      0.90        20
     weighted avg       0.92      0.90      0.90        20



## Task 5 · Classify sample channels
- Build a small list of synthetic channels, convert them with `channel_to_feature`, and stack the results in `X`.
- Use the loaded model to predict labels and review the DataFrame for any surprising cases.

In [5]:
channels = [
    ('depolarizing_p0.1', depolarizing_kraus(0.1)),
    ('depolarizing_p0.5', depolarizing_kraus(0.5)),
    ('amp_damp_0.1', amplitude_damping_kraus(0.1)),
    ('amp_damp_0.5', amplitude_damping_kraus(0.5)),
]

features = []
names = []
for name, ch in channels:
    f = channel_to_feature(ch)
    names.append(name)
    features.append(f)
X_test = np.vstack(features)

preds = model.predict(X_test)
df = pd.DataFrame({'channel': names, 'prediction': preds})
df


,channel,prediction
0,depolarizing_p0.1,amplitude_damping
1,depolarizing_p0.5,depolarizing
2,amp_damp_0.1,amplitude_damping
3,amp_damp_0.5,amplitude_damping


### Submission checklist
- Update `model_path` with the actual artifact you trained in Assignment 3 and note the load result.
- Mention any feature changes you make so the classifier stays compatible with production runs.
- Save this notebook with outputs after running Tasks 1–5 and add a short reflection in your report.